### Fine-Tune Flan T5

- Joel Stremmel
- 04-21-23

##### About

Fine-Tune Flan T5 on the formatted data using K-Fold Cross-Validation and save the scores.  Based on this [blog post](https://www.philschmid.de/fine-tune-flan-t5).

##### Imports

In [1]:
import os
import re
import glob
import pickle
import torch
import random
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EvalPrediction,
    EarlyStoppingCallback
)

2023-04-21 10:33:16.741436: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-21 10:33:16.791096: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-21 10:33:16.791772: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 10:33:17.521322: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##### Set Parameters

In [2]:
max_seq_len = 1024
output_max_seq_len = 2
batch_size = 1
accumulation_steps = 32
lr = 0.00002
weight_decay = 0.01
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_epsilon = 0.00000001
warmup_steps = 4
logging_steps = 1
num_workers = 8
seed = 44
epochs = 5
fp16 = False
colab = False
gradient_checkpointing = False
add_summaries = False
input_dir = './data' # "/content/drive/MyDrive/data/"
model_output_dir = "model_output"
results_dir = './results/' #"/content/drive/MyDrive/results/"
model_key = "flan_t5_small" 
lm_path = "google/flan-t5-small"

##### Optionally Connect to Google Drive

In [3]:
if colab:

    from google.colab import drive
    drive.mount('/content/drive')

##### Disable Tokenizer Parallelism
This is mostly to avoid warnings.

In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

##### Load Formatted Data

In [5]:
if add_summaries:
    
    with open('data/Xwsum_folds.pkl', 'rb') as f:
        X_folds = pickle.load(f)

else:
    
    with open('data/X_folds.pkl', 'rb') as f:
        X_folds = pickle.load(f)

with open('data/y_folds.pkl', 'rb') as f:
    y_folds = pickle.load(f)

##### Check Data Shape

In [6]:
assert len(X_folds) == len(y_folds), "Expected the same number of folds in X and y."
X = list(X_folds.values())
y = list(y_folds.values())

##### Check Target Prevalence

In [7]:
print(f"Target prevalance: {np.mean(np.concatenate(y))}.")

Target prevalance: 0.5166666666666667.


##### Check that GPU is Available

In [8]:
assert torch.cuda.is_available(), "Run this script on a GPU."
print(torch.__version__)

1.8.1+cu101


##### Tokenize Text and Fit Model to Each Fold

In [9]:
y_probs, y_trues = [], []
for i in range(len(X)):
    
    print(f"Fitting model using fold {i} as out of fold data.")
    
    # Identify train folds and shuffle samples
    X_train, y_train = np.concatenate(X[0:i] + X[i+1:], axis=0), np.concatenate(y[0:i] + y[i+1:], axis=0)
    indices = np.arange(len(y_train))
    np.random.shuffle(indices)
    X_train, y_train = X_train[indices], y_train[indices]
    
    # Identify test folds
    X_test, y_test = X[i], y[i]
    
    # Format text and label data as HuggingFace dataset
    train_dataset = Dataset.from_dict({"text": X_train, "label_ids": [str(label) for label in y_train]})
    test_dataset = Dataset.from_dict({"text": X_test, "label_ids": [str(label) for label in y_test]})
    
    # Load model and tokenizer
    # This will reset the model weights with each new iteration
    tokenizer = AutoTokenizer.from_pretrained(lm_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(lm_path)
    
    def preprocess_function(sample, padding="max_length", output_max_seq_len=20):
    
        # Add prefix to the input for t5
        inputs = ["Classify this text as either 1 or 0: " + item for item in sample["text"]]

        # tokenize inputs
        model_inputs = tokenizer(
            inputs,
            max_length=max_seq_len,
            padding=padding,
            truncation=True
        )

        # Tokenize targets with the `text_target` keyword argument
        labels = tokenizer(
            text_target=sample["label_ids"],
            max_length=output_max_seq_len,
            padding=padding,
            truncation=True
        )

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length":
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["label_ids"] = labels["input_ids"]

        return model_inputs
    
    # Tokenize train dataset
    train_dataset = train_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=["text"],
        batch_size=batch_size
    )
    train_dataset.set_format("pt")
    
    # Tokenize test dataset
    test_dataset = test_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=["text"],
        batch_size=batch_size
    )
    test_dataset.set_format("pt")
    
#     eval_steps=PARAMS["eval_steps"],
#     save_steps=PARAMS["save_steps"],
#     evaluation_strategy=PARAMS["evaluation_strategy"],
#     save_strategy=PARAMS["save_strategy"],
#     fp16_full_eval=PARAMS["fp16_eval"],
#     eval_accumulation_steps=PARAMS["eval_accumulation_steps"],
        
    # Define training arguments
    training_args= Seq2SeqTrainingArguments(
        output_dir=model_output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=accumulation_steps,
        warmup_steps=warmup_steps,
        logging_steps=logging_steps,
        weight_decay=weight_decay,
        learning_rate=lr,
        seed=seed,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        dataloader_num_workers=num_workers,
        fp16=fp16,
        logging_strategy="steps",
        save_strategy="no",
        lr_scheduler_type='linear',
        optim="adamw_torch",
        run_name=model_key,
        gradient_checkpointing=gradient_checkpointing,
        generation_max_length=output_max_seq_len,
        predict_with_generate=True,
        generation_num_beams=None,
        do_eval=False,
        fp16_full_eval=False,
        sharded_ddp=False,
        load_best_model_at_end=True,
        prediction_loss_only=False,
        disable_tqdm=True,
        logging_dir=None
    )
    
    # Define model training
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset
    )
    
    # Train model
    trainer.train()
    
    # Predict on test dataset with greedy generation
    output = trainer.predict(test_dataset, do_sample=False, max_length=output_max_seq_len, early_stopping=True)
    preds_decoded = tokenizer.batch_decode(output.predictions, skip_special_tokens=True)
    labels = np.where(output.label_ids != -100, output.label_ids, tokenizer.pad_token_id)
    labels_decoded = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Convert preds to ints
    preds = []
    for pred in preds_decoded:
        if pred == '1':
            preds.append(1)
        elif pred == '0':
            preds.append(0)
        else:
            print(f"Got unexpected pred: {pred}.")
            preds.append(random.choice([0, 1]))

    # Save scores and labels
    y_probs.append(preds)
    y_trues.append([int(label) for label in labels_decoded])

Fitting model using fold 0 as out of fold data.


Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

{'loss': 0.5358, 'learning_rate': 5e-06, 'epoch': 0.68}
{'loss': 0.6231, 'learning_rate': 1e-05, 'epoch': 1.36}
{'loss': 0.5433, 'learning_rate': 1.5000000000000002e-05, 'epoch': 2.04}
{'loss': 0.4672, 'learning_rate': 2e-05, 'epoch': 2.72}
{'loss': 0.6367, 'learning_rate': 0.0, 'epoch': 3.4}
{'train_runtime': 62.621, 'train_samples_per_second': 3.753, 'train_steps_per_second': 0.08, 'train_loss': 0.5612292289733887, 'epoch': 3.4}
Got unexpected pred: .
Got unexpected pred: .
Fitting model using fold 1 as out of fold data.


Map:   0%|          | 0/47 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

{'loss': 0.4758, 'learning_rate': 5e-06, 'epoch': 0.63}
{'loss': 0.5723, 'learning_rate': 1e-05, 'epoch': 1.25}
{'loss': 0.3927, 'learning_rate': 1.5000000000000002e-05, 'epoch': 1.88}
{'loss': 0.4147, 'learning_rate': 2e-05, 'epoch': 2.51}
{'loss': 0.4591, 'learning_rate': 0.0, 'epoch': 3.14}
{'train_runtime': 62.5956, 'train_samples_per_second': 4.074, 'train_steps_per_second': 0.08, 'train_loss': 0.46291311383247374, 'epoch': 3.14}
Got unexpected pred: .
Fitting model using fold 4 as out of fold data.


Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

{'loss': 0.4638, 'learning_rate': 5e-06, 'epoch': 0.68}
{'loss': 0.479, 'learning_rate': 1e-05, 'epoch': 1.36}
{'loss': 0.5766, 'learning_rate': 1.5000000000000002e-05, 'epoch': 2.04}
{'loss': 0.4312, 'learning_rate': 2e-05, 'epoch': 2.72}
{'loss': 0.4, 'learning_rate': 0.0, 'epoch': 3.4}
{'train_runtime': 61.6774, 'train_samples_per_second': 3.81, 'train_steps_per_second': 0.081, 'train_loss': 0.4701070189476013, 'epoch': 3.4}
Got unexpected pred: .


##### Save Model Probabilities on Test Folds and True Labels

In [10]:
with open(os.path.join(results_dir, f'{model_key}_y_trues.pkl'), 'wb') as f:
    pickle.dump(y_trues, f)

with open(os.path.join(results_dir, f'{model_key}_y_probs.pkl'), 'wb') as f:
    pickle.dump(y_probs, f)